# テキストのみで業種予測を予測するコード

In [1]:
!pip install mecab-python3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 9.2 MB/s eta 0:00:00


In [2]:
!pip install unidic
!python -m unidic download

  Preparing metadata (setup.py) ... done
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7403 sha256=2e7faa43d4f73c1b8db58dccbe77ff27fa956159b5a92a93c87b494c4ca8190f
  Stored in directory: /root/.cache/pip/wheels/4c/c6/ae/d243f0d01ea936ab42ee7e820463dcb837079e73764060352d
Successfully built unidic
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.3
    Uninstalling wasabi-1.1.3:
      Successfully uninstalled wasabi-1.1.3
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:16<00:00, 31.8MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.11/dist-packages/unidic/dicdir


In [3]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install japanize-matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=31ad96a964c4c532e9e7f0420da200099de39b6971851e57a34be28aef24c1f6
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib


In [5]:
!pip install umap-learn
import umap

In [6]:
!pip install fugashi
!pip install unidic-lite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.0/698.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=fd85be9f67481d0b12aac6bd0e51fc9723968cc3e8f61c395dc5ecdbaf76f474
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built unidic-lite


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os
import re
import pandas as pd
import unicodedata
from IPython.display import display

In [9]:
# CSVファイルの読み込み
text_file_path1 = "/content/drive/MyDrive/CSV保存用/merged_df_20230401.csv"  # アップロードしたファイルのパス
text_file_path2 = "/content/drive/MyDrive/CSV保存用/merged_df2_v3.csv"
text_file_path3 = "/content/drive/MyDrive/CSV保存用/merged_df4_v2.csv"
text_file_path4 = "/content/drive/MyDrive/CSV保存用/merged_df5_v1.csv"
text_file_path5 = "/content/drive/MyDrive/CSV保存用/merged_df6_v1.csv"
text_file_path6 = "/content/drive/MyDrive/CSV保存用/merged_df7_v1.csv"
text_file_path7 = "/content/drive/MyDrive/CSV保存用/merged_df8_v1.csv"
text_file_path8 = "/content/drive/MyDrive/CSV保存用/merged_df9_v1.csv"

text_df1 = pd.read_csv(text_file_path1)
text_df2 = pd.read_csv(text_file_path2)
text_df3 = pd.read_csv(text_file_path3)
text_df4 = pd.read_csv(text_file_path4)
text_df5 = pd.read_csv(text_file_path5)
text_df6 = pd.read_csv(text_file_path6)
text_df7 = pd.read_csv(text_file_path7)
text_df8 = pd.read_csv(text_file_path8)

#データの結合して重複している行を削除
text_df = pd.concat([text_df1, text_df2, text_df3, text_df4, text_df5, text_df6, text_df7, text_df8], ignore_index=True).drop_duplicates()

# 必要な列を確認して事業内容と経営方針を取得
display(text_df.columns)  # 列名を確認

print(text_df.shape)

# 欠損値を削除
text_df = text_df.dropna(subset=['事業の内容', '経営方針'])

# 事業内容と経営方針を一つのテキストとして結合
text_df['テキスト'] = text_df['事業の内容'] + " " + text_df['経営方針']

#書類名、docID、EDINETコード、決算期、事業の内容、経営方針のカラムを削除
text_df = text_df.drop(columns=['書類名','docID', 'ＥＤＩＮＥＴコード', '決算期'])

print(text_df.shape)
display(text_df.head())

Index(['会社名', '書類名', 'docID', '証券コード', 'ＥＤＩＮＥＴコード', '決算期', '提出日', '提出者業種',
       '事業の内容', '経営方針'],
      dtype='object')

(22305, 10)
(22305, 7)


,会社名,証券コード,提出日,提出者業種,事業の内容,経営方針,テキスト
0,一正蒲鉾株式会社,29040,2024-09-20,食料品,３ 【事業の内容】当社グループは、当社、連結子会社１社及び関連会社１社で構成され、その事業は...,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...,３ 【事業の内容】当社グループは、当社、連結子会社１社及び関連会社１社で構成され、その事業は...
1,株式会社ジョイフル本田,31910,2024-09-20,小売業,３ 【事業の内容】当社は、主に「住まい」に関する関連商品（資材・プロ用品、インテリア・リビン...,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当事業年度末において...,３ 【事業の内容】当社は、主に「住まい」に関する関連商品（資材・プロ用品、インテリア・リビン...
2,株式会社アバントグループ,38360,2024-09-20,情報・通信業,３ 【事業の内容】当社グループは当社及び子会社６社で構成されております。（子会社６社の中には...,１ 【経営方針、経営環境及び対処すべき課題等】文中における将来に関する事項は、有価証券報告書...,３ 【事業の内容】当社グループは当社及び子会社６社で構成されております。（子会社６社の中には...
3,株式会社ブロードバンドセキュリティ,43980,2024-09-20,情報・通信業,３【事業の内容】当社は、企業における情報漏えいの予防や防止、セキュリティ機器の24時間365...,１【経営方針、経営環境及び対処すべき課題等】当社の経営方針、経営環境及び対処すべき課題等は以...,３【事業の内容】当社は、企業における情報漏えいの予防や防止、セキュリティ機器の24時間365...
4,株式会社ユーザーローカル,39840,2024-09-20,情報・通信業,３【事業の内容】（１）事業の概要当社は「ビッグデータ×人工知能で世界を進化させる」という経営...,１【経営方針、経営環境及び対処すべき課題等】当社の経営方針、経営環境及び対処すべき課題等は、...,３【事業の内容】（１）事業の概要当社は「ビッグデータ×人工知能で世界を進化させる」という経営...


In [ ]:
# 会社名が雪印メグミルク株式会社の行を出力
print(text_df[text_df['会社名'] == '雪印メグミルク株式会社'])

               会社名    証券コード         提出日 提出者業種  \
1352   雪印メグミルク株式会社    22700  2024-06-26   食料品   
5144   雪印メグミルク株式会社    22700  2023-06-28   食料品   
9767   雪印メグミルク株式会社  22700.0  2022-06-28   食料品   
13583  雪印メグミルク株式会社  22700.0  2021-06-23   食料品   
16188  雪印メグミルク株式会社  22700.0  2020-06-24   食料品   
20974  雪印メグミルク株式会社  22700.0  2019-06-26   食料品   

                                                   事業の内容  \
1352   ３ 【事業の内容】2024年３月31日現在の当社グループ(当社および当社の関係会社)は、当社...   
5144   ３ 【事業の内容】2023年３月31日現在の当社グループ(当社および当社の関係会社)は、当社...   
9767   ３ 【事業の内容】2022年３月31日現在の当社グループ(当社および当社の関係会社)は、当社...   
13583  ３【事業の内容】2021年３月31日現在の当社グループ（当社および当社の関係会社）は、当社、...   
16188  ３【事業の内容】2020年３月31日現在の当社グループ（当社および当社の関係会社）は、当社、...   
20974  ３【事業の内容】2019年３月31日現在の当社グループ（当社および当社の関係会社）は、当社、...   

                                                    経営方針  \
1352   １ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...   
5144   １ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在...   
9767   １ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在..

In [10]:
def clean_text(text):
    """
    テキストから不要な文字や記号を削除し、整形する関数。
    """
    # 章番号や記号（例: "３ 【事業の内容】" や "１ 【経営方針】"）を削除
    text = re.sub(r'\d+\s*【.*?】', '', text)

    #Unicode正規化
    text = unicodedata.normalize('NFKC', text)
    # 余分な空白や改行を削除
    #text = re.sub(r'\s+', ' ', text)
    #・,「」,()を削除
    text = re.sub('・', '', text)
    text = re.sub('「', '', text)
    text = re.sub('」', '', text)
    text = re.sub(r'\(', '（', text)
    text = re.sub(r'\)', '）', text)
    #(株)→株式会社
    text = text.replace('（株）', '株式会社')
    # 丸括弧で囲まれた番号を削除（例: "(１)"）
    text = re.sub(r'（\d+）', '', text)
    text = re.sub('（', '', text)
    text = re.sub('）', '', text)
    text = re.sub('〔', '', text)
    text = re.sub('〕', '', text)
    text = re.sub(r'\.', '', text)
    text = re.sub(r'『', '', text)
    text = re.sub(r'』', '', text)
    text = re.sub(r'《', '', text)
    text = re.sub(r'》', '', text)
    # 連続するピリオドを削除
    #text = re.sub(r'\.{2,}', '', text)
    # ダブルクオーテーションを削除
    #text = text.replace('"', '')


    # 前後の空白を削除
    return text.strip()

# "事業の内容" と "経営方針" カラムに対して再クリーンアップを適用
text_df['テキスト'] = text_df['テキスト'].apply(clean_text)

# 再度確認用に最初の数行を表示
text_df[['会社名','テキスト']].head()


,会社名,テキスト
0,一正蒲鉾株式会社,当社グループは、当社、連結子会社1社及び関連会社1社で構成され、その事業は、食品の製造販売を...
1,株式会社ジョイフル本田,当社は、主に住まいに関する関連商品資材プロ用品、インテリアリビング、ガーデンファームの販売お...
2,株式会社アバントグループ,当社グループは当社及び子会社6社で構成されております。子会社6社の中には連結注記重要な後発事...
3,株式会社ブロードバンドセキュリティ,当社は、企業における情報漏えいの予防や防止、セキュリティ機器の24時間365日体制での遠隔監...
4,株式会社ユーザーローカル,事業の概要当社はビッグデータ×人工知能で世界を進化させるという経営理念のもと、大量のデジタル...


### SentenceTransformer('stsb-xlm-r-multilingual')でベクトル化

In [ ]:
# SentenceTransformerモデルの読み込み
model = SentenceTransformer('stsb-xlm-r-multilingual')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm  # プログレスバー用ライブラリ
import numpy as np

# 事業内容をリストに変換
business_descriptions = text_df['テキスト'].tolist()

# バッチサイズを設定（100件ずつ処理）
batch_size = 100
num_batches = len(business_descriptions) // batch_size + (1 if len(business_descriptions) % batch_size > 0 else 0)

# ベクトル化の進捗を表示
print("事業内容のベクトル化を開始します...")
business_embeddings = []

# バッチ処理の進捗表示
for batch_idx in tqdm(range(num_batches), desc="ベクトル化の進捗"):
    # バッチデータの抽出
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(business_descriptions))
    batch_descriptions = business_descriptions[batch_start:batch_end]

    # バッチを一度にベクトル化
    batch_embeddings = model.encode(batch_descriptions)

    # 結果を格納
    business_embeddings.extend(batch_embeddings)

print("ベクトル化が完了しました。")


事業内容のベクトル化を開始します...


ベクトル化の進捗: 100%|██████████| 224/224 [04:19<00:00,  1.16s/it]

ベクトル化が完了しました。


In [ ]:
#クラスタリングなし
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# 業種ラベルを数値にエンコード
label_encoder = LabelEncoder()
text_df['提出者業種_encoded'] = label_encoder.fit_transform(text_df['提出者業種'])

# ベクトル化済み事業内容と業種ラベルを取得
X = business_embeddings  # ベクトル化された事業内容
y = text_df['提出者業種_encoded']  # エンコードされた業種ラベル

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost分類器の構築と訓練
clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

# テストデータで予測
y_pred = clf.predict(X_test)

# 結果の表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8697601434655907


In [ ]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.936364,0.786260,0.854772,131.00000
その他金融業,0.979167,0.854545,0.912621,55.00000
ガラス・土石製品,0.912281,0.838710,0.873950,62.00000
ゴム製品,0.958333,0.793103,0.867925,29.00000
サービス業,0.805970,0.882353,0.842434,612.00000
パルプ・紙,0.850000,0.708333,0.772727,24.00000
不動産業,0.862745,0.776471,0.817337,170.00000
保険業,1.000000,0.730769,0.844444,26.00000
倉庫・運輸関連,0.940000,0.824561,0.878505,57.00000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.954545,0.617647,0.750000,34.00000


正確率: 0.87


In [ ]:
#SentenceTransformer + k-means
from sklearn.cluster import KMeans

# ラベルエンコーディング（業種を数値に変換）
label_encoder = LabelEncoder()
text_df["業種ラベル"] = label_encoder.fit_transform(text_df["提出者業種"])

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
text_df["クラスタ"] = kmeans.fit_predict(business_embeddings)

X = np.hstack([business_embeddings, text_df["クラスタ"].values.reshape(-1, 1)])  # ベクトル+クラスタ番号
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8663976686841516


In [ ]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.929204,0.801527,0.860656,131.000000
その他金融業,0.938776,0.836364,0.884615,55.000000
ガラス・土石製品,0.946429,0.854839,0.898305,62.000000
ゴム製品,0.958333,0.793103,0.867925,29.000000
サービス業,0.786765,0.874183,0.828173,612.000000
パルプ・紙,0.818182,0.750000,0.782609,24.000000
不動産業,0.842105,0.752941,0.795031,170.000000
保険業,1.000000,0.692308,0.818182,26.000000
倉庫・運輸関連,0.957447,0.789474,0.865385,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.954545,0.617647,0.750000,34.000000


正確率: 0.87


In [ ]:
#SentenceTransformer + GMM
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=num_clusters, random_state=42)
text_df["クラスタ"] = gmm.fit_predict(business_embeddings)

# クラスタの確率分布を特徴量として追加
gmm_probs = gmm.predict_proba(business_embeddings)  # 各クラスタの所属確率

X = np.hstack([business_embeddings, text_df["クラスタ"].values.reshape(-1, 1), gmm_probs])
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8679668235821565


In [ ]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.929825,0.809160,0.865306,131.000000
その他金融業,0.938776,0.836364,0.884615,55.000000
ガラス・土石製品,0.946429,0.854839,0.898305,62.000000
ゴム製品,0.958333,0.793103,0.867925,29.000000
サービス業,0.797654,0.888889,0.840804,612.000000
パルプ・紙,0.818182,0.750000,0.782609,24.000000
不動産業,0.820513,0.752941,0.785276,170.000000
保険業,1.000000,0.769231,0.869565,26.000000
倉庫・運輸関連,0.916667,0.771930,0.838095,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,1.000000,0.558824,0.716981,34.000000


正確率: 0.87


In [ ]:
#SentenceTransformer + LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(text_df["テキスト"])

# 提出者業種の種類の数を取得（LDAのトピック数に設定）
num_topics = text_df["提出者業種"].nunique()

lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
topic_distributions = lda.fit_transform(tfidf_matrix)  # 各文書のトピック確率分布

X = np.hstack([business_embeddings, topic_distributions])  # ベクトル + LDA のトピック分布
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8755884330867518


### SentenceTransformer('sonoisa/sentence-bert-base-ja-mean-tokens-v2')でベクトル化

In [11]:
model = SentenceTransformer('sonoisa/sentence-bert-base-ja-mean-tokens-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from tqdm import tqdm  # プログレスバー用ライブラリ
import numpy as np

# 事業内容をリストに変換
business_descriptions = text_df['テキスト'].tolist()

# バッチサイズを設定（100件ずつ処理）
batch_size = 100
num_batches = len(business_descriptions) // batch_size + (1 if len(business_descriptions) % batch_size > 0 else 0)

# ベクトル化の進捗を表示
print("事業内容のベクトル化を開始します...")
business_embeddings = []

# バッチ処理の進捗表示
for batch_idx in tqdm(range(num_batches), desc="ベクトル化の進捗"):
    # バッチデータの抽出
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(business_descriptions))
    batch_descriptions = business_descriptions[batch_start:batch_end]

    # バッチを一度にベクトル化
    batch_embeddings = model.encode(batch_descriptions)

    # 結果を格納
    business_embeddings.extend(batch_embeddings)

print("ベクトル化が完了しました。")


事業内容のベクトル化を開始します...


ベクトル化の進捗: 100%|██████████| 224/224 [09:35<00:00,  2.57s/it]

ベクトル化が完了しました。


In [13]:
# クラスタリングのみ
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# 業種ラベルを数値にエンコード
label_encoder = LabelEncoder()
text_df['提出者業種_encoded'] = label_encoder.fit_transform(text_df['提出者業種'])

# ベクトル化済み事業内容と業種ラベルを取得
X = business_embeddings  # ベクトル化された事業内容
y = text_df['提出者業種_encoded']  # エンコードされた業種ラベル

# X に text_df のインデックスを設定
X = pd.DataFrame(X, index=text_df.index)

# 訓練データとテストデータに分割 (インデックスを保持)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練データとテストデータのインデックスを取得
train_indices = X_train.index
test_indices = X_test.index

# XGBoost分類器の構築と訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric="mlogloss")
clf.fit(X_train, y_train)

# テストデータで予測
y_pred = clf.predict(X_test)

# 結果の表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8825375476350594


In [14]:
from sklearn.metrics import classification_report
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.872881,0.786260,0.827309,131.000000
その他金融業,0.921569,0.854545,0.886792,55.000000
ガラス・土石製品,0.980392,0.806452,0.884956,62.000000
ゴム製品,1.000000,0.793103,0.884615,29.000000
サービス業,0.801449,0.903595,0.849462,612.000000
パルプ・紙,1.000000,0.791667,0.883721,24.000000
不動産業,0.931507,0.800000,0.860759,170.000000
保険業,0.909091,0.769231,0.833333,26.000000
倉庫・運輸関連,0.943396,0.877193,0.909091,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.960000,0.705882,0.813559,34.000000


正確率: 0.88


In [15]:
# 誤予測したデータをCSVで出力

# 誤予測したデータのインデックスを取得
misclassified_indices = y_test.index[y_pred != y_test.values]

# 誤予測したデータの会社名とテキストを抽出
misclassified_companies = text_df.loc[misclassified_indices, '会社名'].values
misclassified_texts = text_df.loc[misclassified_indices, 'テキスト'].values

# 誤予測のブールマスクを作成
misclassified_mask = y_pred != y_test.values  # 誤予測データのブールマスク

# 予測ラベルと正解ラベルを取得
predicted_labels = label_encoder.inverse_transform(y_pred[misclassified_mask])
true_labels = label_encoder.inverse_transform(y_test.values[misclassified_mask])

# 長さの一致をチェック
assert len(misclassified_companies) == len(misclassified_texts) == len(predicted_labels) == len(true_labels), "リストの長さが一致しません"

# CSV用のデータを作成
misclassified_data = {
    '会社名': misclassified_companies,
    'テキスト': misclassified_texts,
    '予測ラベル': predicted_labels,
    '正解ラベル': true_labels,
}

# DataFrame に変換して保存
misclassified_df = pd.DataFrame(misclassified_data)
misclassified_df.to_csv('/content/drive/My Drive/misclassified_predictions_n1.csv', index=False, encoding='utf-8-sig')

print("誤予測データを CSV に保存しました。")


誤予測データを CSV に保存しました。


In [ ]:
# sonoisa + k-means
from sklearn.cluster import KMeans

# ラベルエンコーディング（業種を数値に変換）
label_encoder = LabelEncoder()
text_df["業種ラベル"] = label_encoder.fit_transform(text_df["提出者業種"])

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
text_df["クラスタ"] = kmeans.fit_predict(business_embeddings)

X = np.hstack([business_embeddings, text_df["クラスタ"].values.reshape(-1, 1)])  # ベクトル+クラスタ番号
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8827617126204886


In [ ]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.871795,0.778626,0.822581,131.000000
その他金融業,0.901961,0.836364,0.867925,55.000000
ガラス・土石製品,0.925926,0.806452,0.862069,62.000000
ゴム製品,1.000000,0.758621,0.862745,29.000000
サービス業,0.809316,0.908497,0.856043,612.000000
パルプ・紙,1.000000,0.833333,0.909091,24.000000
不動産業,0.921053,0.823529,0.869565,170.000000
保険業,0.913043,0.807692,0.857143,26.000000
倉庫・運輸関連,0.941176,0.842105,0.888889,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.958333,0.676471,0.793103,34.000000


正確率: 0.88


In [ ]:
# sonoisa + GMM
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=num_clusters, random_state=42)
text_df["クラスタ"] = gmm.fit_predict(business_embeddings)

# クラスタの確率分布を特徴量として追加
gmm_probs = gmm.predict_proba(business_embeddings)  # 各クラスタの所属確率

X = np.hstack([business_embeddings, text_df["クラスタ"].values.reshape(-1, 1), gmm_probs])
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8852275274602107


In [ ]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.850877,0.740458,0.791837,131.000000
その他金融業,0.872727,0.872727,0.872727,55.000000
ガラス・土石製品,1.000000,0.790323,0.882883,62.000000
ゴム製品,1.000000,0.793103,0.884615,29.000000
サービス業,0.832073,0.898693,0.864101,612.000000
パルプ・紙,0.950000,0.791667,0.863636,24.000000
不動産業,0.896774,0.817647,0.855385,170.000000
保険業,0.913043,0.807692,0.857143,26.000000
倉庫・運輸関連,0.938776,0.807018,0.867925,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,1.000000,0.676471,0.807018,34.000000


正確率: 0.89


In [ ]:
# sonoisa + LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(text_df["テキスト"])

# 提出者業種の種類の数を取得（LDAのトピック数に設定）
num_topics = text_df["提出者業種"].nunique()

lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
topic_distributions = lda.fit_transform(tfidf_matrix)  # 各文書のトピック確率分布

X = np.hstack([business_embeddings, topic_distributions])  # ベクトル + LDA のトピック分布
y = text_df["業種ラベル"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8823133826496301


### TF-IDF（従来手法）で学習

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF ベクトル化
vectorizer = TfidfVectorizer(stop_words=None, max_features=5000)
X_tfidf = vectorizer.fit_transform(text_df['テキスト'])

In [17]:
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

# MeCabの準備（辞書は環境に合わせて変更）
mecab = MeCab.Tagger("-Owakati")  # 分かち書き

# 事業内容を単語単位に分割
tokenized_texts = text_df['テキスト'].apply(lambda text: mecab.parse(text).strip())

# TF-IDF ベクトル化
vectorizer = TfidfVectorizer(stop_words=None, max_features=5000)
X_tfidf = vectorizer.fit_transform(tokenized_texts)


In [18]:
# ラベルエンコーディング（業種を数値に変換）
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
text_df["業種ラベル"] = label_encoder.fit_transform(text_df["提出者業種"])

In [19]:
# クラスタリングなし
X = X_tfidf  # TF-IDF でテキストをベクトル化
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9495628782784129


In [20]:
#TF-IDF + K-means
from sklearn.cluster import KMeans

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

# K-means クラスタリング
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_tfidf)

# クラスタ番号を特徴量として追加
text_df["cluster"] = cluster_labels

# 学習データの作成（TF-IDF + クラスタ情報）
X = pd.concat([pd.DataFrame(X_tfidf.toarray()).reset_index(drop=True), pd.DataFrame(text_df["cluster"]).reset_index(drop=True)], axis=1, ignore_index=True)
y = text_df["業種ラベル"]

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = xgb.XGBClassifier(random_state=42,eval_metric='mlogloss', n_jobs=-1, tree_method='hist', device='cuda')
clf.fit(X_train, y_train)

# GPU にデータをコピーして予測
X_test_gpu = xgb.DMatrix(X_test, device='cuda')
y_pred = clf.predict(X_test_gpu)

print("Accuracy:", accuracy_score(y_test, y_pred))

TypeError: DMatrix.__init__() got an unexpected keyword argument 'device'

In [21]:
#TF-IDF + GMM
from sklearn.decomposition import TruncatedSVD
from sklearn.mixture import GaussianMixture
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import cupy as cp

# TF-IDF の次元削減（300次元に圧縮）
svd = TruncatedSVD(n_components=300, random_state=42)
X_reduced = svd.fit_transform(X_tfidf)

# GMM クラスタリング
num_clusters = text_df["提出者業種"].nunique()  # クラスタ数
gmm = GaussianMixture(n_components=num_clusters, covariance_type='full', random_state=42)

cluster_labels = gmm.fit_predict(X_reduced)  # クラスタリング実行

# クラスタ情報をデータに追加
text_df["cluster"] = cluster_labels

# 学習データの作成（TF-IDF + クラスタ情報）
X = pd.concat([pd.DataFrame(X_tfidf.toarray()).reset_index(drop=True),
               pd.DataFrame(text_df["cluster"]).reset_index(drop=True)], axis=1, ignore_index=True)
y = text_df["業種ラベル"]

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost の分類モデルを定義（GPU を使用）
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1, tree_method='hist', device='cuda')

# モデルの学習
clf.fit(X_train, y_train)

# GPU にデータをコピー
X_test_gpu = cp.array(X_test)  # NumPy → CuPy に変換

# 予測時に DMatrix を使用せず、CuPy 配列を直接渡す
y_pred = clf.predict(X_test_gpu.get())  # CuPy 配列を NumPy 配列に変換して予測

# 精度の表示
print("Accuracy:", accuracy_score(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [01:31:00] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.9506837032055593


In [22]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.933333,0.854962,0.892430,131.000000
その他金融業,1.000000,1.000000,1.000000,55.000000
ガラス・土石製品,0.949153,0.903226,0.925620,62.000000
ゴム製品,0.903226,0.965517,0.933333,29.000000
サービス業,0.927885,0.946078,0.936893,612.000000
パルプ・紙,0.920000,0.958333,0.938776,24.000000
不動産業,0.964706,0.964706,0.964706,170.000000
保険業,0.866667,1.000000,0.928571,26.000000
倉庫・運輸関連,0.964912,0.964912,0.964912,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.952381,0.588235,0.727273,34.000000


正確率: 0.95


In [23]:
#TF-IDF + LDA
from sklearn.decomposition import LatentDirichletAllocation

# トピック数（提出者業種の種類の数に設定）
num_topics = text_df["提出者業種"].nunique()

# LDA によるトピックモデリング
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
topic_distributions = lda.fit_transform(X_tfidf.toarray())  # 各文書のトピック分布

# トピック分布を特徴量として追加
topic_df = pd.DataFrame(topic_distributions)

# 学習データの作成（TF-IDF + LDA トピック情報）
X = pd.concat([pd.DataFrame(X_tfidf.toarray()).reset_index(drop=True),
               topic_df.reset_index(drop=True)], axis=1, ignore_index=True)
y = text_df["業種ラベル"]

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost の分類モデルを定義（GPU を使用）
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss', n_jobs=-1, tree_method='hist', device='cuda')

# モデルの学習
clf.fit(X_train, y_train)

# GPU にデータをコピー
X_test_gpu = cp.array(X_test)  # NumPy → CuPy に変換

# 予測時に DMatrix を使用せず、CuPy 配列を直接渡す
y_pred = clf.predict(X_test_gpu.get())  # CuPy 配列を NumPy 配列に変換して予測

# 精度の表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9538220130015691


In [24]:
# classification_reportを辞書形式で取得
report_dict = classification_report(
    y_test,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0,
    output_dict=True
)

# データフレームに変換
report_df = pd.DataFrame(report_dict).transpose()

# 正確率の計算
accuracy = accuracy_score(y_test, y_pred)

# 結果の表示
print("="*50)
print("分類結果 (表形式):")
print("="*50)

# classification_reportの結果を表示
display(report_df)

# 正確率の表示
print("="*50)
print(f"正確率: {accuracy:.2f}")
print("="*50)

分類結果 (表形式):


,precision,recall,f1-score,support
その他製品,0.941667,0.862595,0.900398,131.000000
その他金融業,0.981818,0.981818,0.981818,55.000000
ガラス・土石製品,0.950000,0.919355,0.934426,62.000000
ゴム製品,0.933333,0.965517,0.949153,29.000000
サービス業,0.931142,0.972222,0.951239,612.000000
パルプ・紙,1.000000,0.958333,0.978723,24.000000
不動産業,0.976048,0.958824,0.967359,170.000000
保険業,1.000000,1.000000,1.000000,26.000000
倉庫・運輸関連,0.964286,0.947368,0.955752,57.000000
内国法人・組合（有価証券報告書等の提出義務者以外）,0.941176,0.470588,0.627451,34.000000


正確率: 0.95


### Word2vecでベクトル化

In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm
import MeCab

# MeCabの準備（辞書は環境に合わせて変更）
mecab = MeCab.Tagger("-Owakati")  # 分かち書き

# 事業内容をリストに変換
business_descriptions = text_df['テキスト'].tolist()

# テキストを単語単位に分割
tokenized_descriptions = [mecab.parse(text).strip().split() for text in business_descriptions]

# バッチサイズを設定（100件ずつ処理）
batch_size = 100
num_batches = len(tokenized_descriptions) // batch_size + (1 if len(tokenized_descriptions) % batch_size > 0 else 0)

# Word2Vecモデルを訓練（事前に訓練したWord2Vecモデルを使うことも可能）
word2vec_model = Word2Vec(tokenized_descriptions, vector_size=200, window=10, min_count=5, workers=4, sg=0, negative=10, epochs=20)

# ベクトル化の進捗を表示
print("事業内容のベクトル化を開始します...")
X_word2vec = []

# バッチ処理の進捗表示
for batch_idx in tqdm(range(num_batches), desc="ベクトル化の進捗"):
    # バッチデータの抽出
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(tokenized_descriptions))
    batch_descriptions = tokenized_descriptions[batch_start:batch_end]

    # バッチを一度にベクトル化
    batch_embeddings = []
    for description in batch_descriptions:
        vector = np.zeros(word2vec_model.vector_size)
        num_words = 0
        for word in description:
            if word in word2vec_model.wv:
                vector += word2vec_model.wv[word]
                num_words += 1
        if num_words > 0:
            vector /= num_words
        batch_embeddings.append(vector)

    # 結果を格納
    X_word2vec.extend(batch_embeddings)

print("ベクトル化が完了しました。")


In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm

# 事業内容をリストに変換
business_descriptions = text_df['テキスト'].tolist()

# バッチサイズを設定（100件ずつ処理）
batch_size = 100
num_batches = len(business_descriptions) // batch_size + (1 if len(business_descriptions) % batch_size > 0 else 0)

# Word2Vecモデルを訓練（事前に訓練したWord2Vecモデルを使うことも可能）
sentences = [text.split() for text in business_descriptions]
word2vec_model = Word2Vec(sentences, vector_size=200, window=10, min_count=5, workers=4, sg=0, negative=10, epochs=20)

# ベクトル化の進捗を表示
print("事業内容のベクトル化を開始します...")
X_word2vec = []

# バッチ処理の進捗表示
for batch_idx in tqdm(range(num_batches), desc="ベクトル化の進捗"):
    # バッチデータの抽出
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(business_descriptions))
    batch_descriptions = business_descriptions[batch_start:batch_end]

    # バッチを一度にベクトル化
    batch_embeddings = []
    for description in batch_descriptions:
        vector = np.zeros(word2vec_model.vector_size)
        num_words = 0
        for word in description.split():
            if word in word2vec_model.wv:
                vector += word2vec_model.wv[word]
                num_words += 1
        if num_words > 0:
            vector /= num_words
        batch_embeddings.append(vector)

    # 結果を格納
    X_word2vec.extend(batch_embeddings)

print("ベクトル化が完了しました。")


事業内容のベクトル化を開始します...


ベクトル化の進捗: 100%|██████████| 133/133 [00:00<00:00, 143.43it/s]

ベクトル化が完了しました。


In [ ]:
# ラベルエンコーディング（業種を数値に変換）
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
text_df["業種ラベル"] = label_encoder.fit_transform(text_df["提出者業種"])

In [ ]:
# クラスタリングなし
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_word2vecを使用（事業内容をWord2Vecでベクトル化した結果）
X = X_word2vec  # Word2Vec でベクトル化
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoostの分類器を訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# 予測
y_pred = clf.predict(X_test)

# 精度を表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8713292983635956


In [ ]:
#Word2vec + k-means
# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

# K-means クラスタリング
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_word2vec)  # Word2Vecで得られたベクトルをクラスタリング

# クラスタ番号を特徴量として追加
text_df["cluster"] = cluster_labels

# 学習データの作成（Word2Vecベクトル + クラスタ情報）
# クラスタラベルを特徴量として追加
X_with_clusters = np.hstack([X, cluster_labels.reshape(-1, 1)])
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_with_clusters, y, test_size=0.2, random_state=42)

# XGBoostの分類器を訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# 精度を表示
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8713292983635956


In [ ]:
#Word2vec + GMM
from sklearn.mixture import GaussianMixture
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

# GMM クラスタリング
gmm = GaussianMixture(n_components=num_clusters, random_state=42)
cluster_labels = gmm.fit_predict(X_word2vec)  # Word2Vecで得られたベクトルをクラスタリング

# クラスタ番号を特徴量として追加
text_df["cluster"] = cluster_labels

# 学習データの作成（Word2Vecベクトル + クラスタ情報）
X_with_clusters = np.hstack([X_word2vec, cluster_labels.reshape(-1, 1)])
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_with_clusters, y, test_size=0.2, random_state=42)

# XGBoostの分類器を訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# テストデータに対して予測
y_pred = clf.predict(X_test)

# 精度を表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8762609280430397


In [ ]:
#Word2vec + GMM
from sklearn.mixture import GaussianMixture
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

# GMM クラスタリング
gmm = GaussianMixture(n_components=num_clusters, random_state=42)
gmm.fit(X_word2vec)

cluster_probs = gmm.predict_proba(X_word2vec)


# 学習データの作成（Word2Vecベクトル + クラスタ情報）
X_with_clusters = np.hstack([X_word2vec, cluster_probs])
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_with_clusters, y, test_size=0.2, random_state=42)

# XGBoostの分類器を訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# テストデータに対して予測
y_pred = clf.predict(X_test)

# 精度を表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8735709482178884


In [ ]:
#Word2vec + LDA
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 提出者業種の種類の数を取得（クラスター数に設定）
num_clusters = text_df["提出者業種"].nunique()

# ReLUを適用して非負化
X_word2vec_relu = np.maximum(0, X_word2vec)  # 負の値をゼロに変換

# LDAによるクラスタリング（Latent Dirichlet Allocation）
lda = LatentDirichletAllocation(n_components=num_clusters, random_state=42)
lda_topic_distributions = lda.fit_transform(X_word2vec_relu)  # 非負化されたWord2VecをLDAにかける

# LDAのトピック分布をクラスタ番号として特徴量に追加
cluster_labels = np.argmax(lda_topic_distributions, axis=1)  # トピック分布から最大のトピックを選択
text_df["cluster"] = cluster_labels

# 学習データの作成（Word2Vecベクトル + クラスタ情報）
X_with_clusters = np.hstack([X_word2vec, cluster_labels.reshape(-1, 1)])
y = text_df["業種ラベル"]  # ラベル（業種）

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_with_clusters, y, test_size=0.2, random_state=42)

# XGBoostの分類器を訓練
clf = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')
clf.fit(X_train, y_train)

# テストデータに対して予測
y_pred = clf.predict(X_test)

# 精度を表示
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8722259583053127
